In [9]:
# assign some values to the vector X
grades_array = ["A", "B", "C", "D", "E"]
# Write a grade mutating function
function topstudent!(x)
	x[4] = "AAAA"
end

println("Your previous grades were $(grades_array[4])")

# Call the function on grades_array
topstudent!(grades_array)

println("Your new grades are $(grades_array[4])")

Your previous grades were D
Your new grades are AAAA


In [10]:
# non mutating function, more expensive

grades_array2 = ["A", "B", "C", "D", "E"]

function topstudent(x)
	x[4] = "AAAA"
    return x # this is a new copy of x
end

println("Your previous grades were $(grades_array2[4])")

grades_array2 = topstudent(grades_array2)

println("Your new grades are $(grades_array2[4])")


Your previous grades were AAAA
Your new grades are AAAA
